# Modelagem Camada Gold

In [0]:
%sql
CREATE VOLUME IF NOT EXISTS workspace.default.gold_accident

In [0]:
%sql
CREATE VOLUME IF NOT EXISTS workspace.default.gold_dim_tempo;
CREATE VOLUME IF NOT EXISTS workspace.default.gold_dim_localizacao;
CREATE VOLUME IF NOT EXISTS workspace.default.gold_dim_acidente;
CREATE VOLUME IF NOT EXISTS workspace.default.gold_fato_acidente;

In [0]:
silver_path = "/Volumes/workspace/default/silver_accident"
df_silver = spark.read.format("delta").load(silver_path)
print("Silver rows:", df_silver.count())

Silver rows: 463152


### Dim_Tempo

In [0]:
from pyspark.sql.functions import monotonically_increasing_id

dim_tempo = (
    df_silver
    .select(
        "data_inversa",
        "ano",
        "mes",
        "dia",
        "dia_semana"
    )
    .dropDuplicates()
    .withColumn("id_tempo", monotonically_increasing_id())
)
display(dim_tempo.limit(10))

data_inversa,ano,mes,dia,dia_semana,id_tempo
2017-02-05,2017,2,5,domingo,0
2017-02-11,2017,2,11,sábado,1
2017-02-12,2017,2,12,domingo,2
2017-02-26,2017,2,26,domingo,3
2017-02-20,2017,2,20,segunda-feira,4
2017-02-10,2017,2,10,sexta-feira,5
2017-01-18,2017,1,18,quarta-feira,6
2017-01-15,2017,1,15,domingo,7
2017-03-07,2017,3,7,terça-feira,8
2017-02-09,2017,2,9,quinta-feira,9


In [0]:
dim_tempo_path = "/Volumes/workspace/default/gold_dim_tempo"
(
    dim_tempo
    .write
    .format("delta")
    .mode("overwrite")
    .save(dim_tempo_path)
)

In [0]:
spark.read.format("delta").load(dim_tempo_path).count()

2434

### Dim_Localizacao

In [0]:
dim_localizacao = (
    df_silver
    .select(
        "uf",
        "br",
        "km",
        "municipio",
        "latitude",
        "longitude",
        "regional",
        "delegacia"
    )
    .dropDuplicates()
    .withColumn("id_localizacao", monotonically_increasing_id())
)

display(dim_localizacao.limit(10))

uf,br,km,municipio,latitude,longitude,regional,delegacia,id_localizacao
PE,101,17,GOIANA,-7.6411,-34.9557,SPRF-PE,DEL01-PE,0
PR,277,36,MORRETES,-25.57444,-48.84733,SPRF-PR,DEL01-PR,1
RO,364,"57,6",VILHENA,-12.52025,-60.40972,SPRF-RO,DEL04-RO,2
RS,290,"384,1",SANTA MARGARIDA DO SUL,-30.3482,-53.99253,SPRF-RS,DEL09-RS,3
PR,163,250,TOLEDO,-24.69,-53.7479,SPRF-PR,DEL06-PR,4
SC,101,"24,2",JOINVILLE,-26.17598,-48.90798,SPRF-SC,DEL03-SC,5
GO,40,17,LUZIANIA,-16.22736,-47.9139,SPRF-DF,DEL01-DF,6
PR,369,"21,7",CAMBARA,-23.04082,-50.09715,SPRF-PR,DEL07-PR,7
RS,293,"16,3",CAPAO DO LEAO,-31.74737,-52.48607,SPRF-RS,DEL07-RS,8
MS,163,"530,6",JARAGUARI,-20.0677,-54.42463,SPRF-MS,DEL01-MS,9


In [0]:
dim_localizacao_path = "/Volumes/workspace/default/gold_dim_localizacao"

(
    dim_localizacao
    .write
    .format("delta")
    .mode("overwrite")
    .save(dim_localizacao_path)
)

### Dim_Acidente

In [0]:
dim_acidente = (
    df_silver
    .select(
        "causa_acidente",
        "tipo_acidente",
        "classificacao_acidente",
        "fase_dia",
        "sentido_via",
        "condicao_metereologica",
        "tipo_pista",
        "tracado_via"
    )
    .dropDuplicates()
    .withColumn("id_caracteristica", monotonically_increasing_id())
)

display(dim_acidente.limit(10))

causa_acidente,tipo_acidente,classificacao_acidente,fase_dia,sentido_via,condicao_metereologica,tipo_pista,tracado_via,id_caracteristica
Condutor Dormindo,Colisão traseira,Com Vítimas Feridas,Plena Noite,Crescente,Céu Claro,Dupla,Não Informado,0
Velocidade Incompatível,Capotamento,Com Vítimas Feridas,Anoitecer,Decrescente,Céu Claro,Simples,Curva,1
Falta de Atenção à Condução,Colisão traseira,Sem Vítimas,Pleno dia,Decrescente,Nublado,Simples,Reta,2
Condutor Dormindo,Colisão transversal,Com Vítimas Feridas,Pleno dia,Crescente,Céu Claro,Simples,Curva,3
Falta de Atenção à Condução,Colisão lateral,Com Vítimas Feridas,Pleno dia,Decrescente,Céu Claro,Múltipla,Reta,4
Velocidade Incompatível,Colisão com objeto estático,Sem Vítimas,Anoitecer,Decrescente,Chuva,Dupla,Curva,5
Deficiência ou não Acionamento do Sistema de Iluminação/Sinalização do Veículo,Colisão com objeto estático,Sem Vítimas,Plena Noite,Crescente,Ignorado,Simples,Reta,6
Ultrapassagem Indevida,Colisão lateral,Com Vítimas Feridas,Plena Noite,Crescente,Céu Claro,Simples,Reta,7
Falta de Atenção à Condução,Colisão traseira,Com Vítimas Feridas,Anoitecer,Decrescente,Nublado,Dupla,Reta,8
Pista Escorregadia,Saída de leito carroçável,Com Vítimas Feridas,Pleno dia,Crescente,Chuva,Simples,Não Informado,9


In [0]:
dim_acidente_path = "/Volumes/workspace/default/gold_dim_acidente"

(
    dim_acidente
    .write
    .format("delta")
    .mode("overwrite")
    .save(dim_acidente_path)
)

### Fato_Acidente (JOIN)

In [0]:
fato_acidente = (
    df_silver
    .join(dim_tempo, ["data_inversa", "ano", "mes", "dia", "dia_semana"], "left")
    .join(dim_localizacao, ["uf", "br", "km", "municipio", "latitude", "longitude", "regional", "delegacia"], "left")
    .join(dim_acidente, [
        "causa_acidente",
        "tipo_acidente",
        "classificacao_acidente",
        "fase_dia",
        "sentido_via",
        "condicao_metereologica",
        "tipo_pista",
        "tracado_via"
    ], "left")
    .select(
        "id_tempo",
        "id_localizacao",
        "id_caracteristica",
        "pessoas",
        "mortos",
        "feridos",
        "feridos_graves",
        "feridos_leves",
        "ilesos",
        "veiculos"
    )
)
display(fato_acidente.limit(10))

id_tempo,id_localizacao,id_caracteristica,pessoas,mortos,feridos,feridos_graves,feridos_leves,ilesos,veiculos
1374,42949684264,25769836452,6.0,0.0,4.0,0.0,4.0,2.0,2.0
1374,60129578962,13974,2.0,1.0,0.0,0.0,0.0,1.0,2.0
1374,25769839319,25769816060,5.0,1.0,2.0,1.0,1.0,2.0,2.0
1374,51539611527,17179881001,4.0,1.0,0.0,0.0,0.0,3.0,3.0
1374,17179884273,17179883368,3.0,0.0,3.0,1.0,2.0,0.0,1.0
1374,8589968642,25769828465,2.0,1.0,0.0,0.0,0.0,1.0,2.0
1374,25769826491,25769838738,5.0,0.0,1.0,0.0,1.0,4.0,2.0
1374,51539638164,25769843325,2.0,0.0,0.0,0.0,0.0,2.0,2.0
1374,51539620720,8589944517,3.0,0.0,3.0,0.0,3.0,0.0,1.0
1374,8589940694,25769828465,2.0,1.0,0.0,0.0,0.0,1.0,2.0


In [0]:
fato_acidente_path = "/Volumes/workspace/default/gold_fato_acidente"

(
    fato_acidente
    .write
    .format("delta")
    .mode("overwrite")
    .save(fato_acidente_path)
)


In [0]:
print("Fato rows:", spark.read.format("delta").load(fato_acidente_path).count())

Fato rows: 463152
